In [1]:
pip install yfinance --upgrade --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install alpha-vantage

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import relevant libraries
import numpy as np
import pandas as pd
import yfinance as yf
import requests
from datetime import datetime, timedelta
from scipy.stats.mstats import winsorize

In [4]:
# Set tickers
ticker_target = 'NVDA'
ticker_comp = 'AMD 005930.KS' # AMD, Samsung
ticker_supp = 'TSM ASML 000660.KS MU' # TSMC, ASML, SK Hynix
ticker_other = 'AMZN MSFT GOOG' # Amazon, Microsoft, Google
ticker_mkt = '^IXIC ^GSPC ^SOX'# Indices - Nasdaq composite, S&P500, PHLX Semiconductor
ticker_all  = " ".join([ticker_target, ticker_comp, ticker_supp, ticker_other, ticker_mkt])

# Print to check 
print(ticker_all)

NVDA AMD 005930.KS TSM ASML 000660.KS MU AMZN MSFT GOOG ^IXIC ^GSPC ^SOX


In [5]:
# Set end date
today = datetime.today().date()
end_date = today.strftime('%Y-%m-%d')

# Set start date
start = datetime(2024, 5, 6).date()
start_date = start.strftime('%Y-%m-%d')

# Print to check
print(end_date)
print(start_date)

2024-05-16
2024-05-06


In [6]:
# Load stock data into a DataFrame
df = yf.download(ticker_all, start=start_date, end=end_date)

# Print to check
print(df)
print(df.info())

[*********************100%%**********************]  13 of 13 completed

Price      Adj Close                                                \
Ticker     000660.KS 005930.KS         AMD        AMZN        ASML   
Date                                                                 
2024-05-06       NaN       NaN  155.779999  188.699997  916.919983   
2024-05-07  179600.0   81300.0  154.429993  188.759995  908.219971   
2024-05-08  178000.0   81300.0  153.619995  188.000000  911.469971   
2024-05-09  175400.0   79700.0  152.389999  189.500000  913.539978   
2024-05-10  179900.0   79200.0  151.919998  187.479996  930.289978   
2024-05-13  183800.0   78400.0  150.559998  186.570007  917.239990   
2024-05-14  185300.0   78300.0  153.160004  187.070007  915.030029   
2024-05-15       NaN       NaN  159.669998  185.990005  937.419983   

Price                                                                   ...  \
Ticker            GOOG        MSFT          MU        NVDA         TSM  ...   
Date                                                                   

In [7]:
# Create new DataFrame with only close prices
new_df = df['Adj Close']

# Print to check
print(new_df)
print(new_df.info())

Ticker      000660.KS  005930.KS         AMD        AMZN        ASML  \
Date                                                                   
2024-05-06        NaN        NaN  155.779999  188.699997  916.919983   
2024-05-07   179600.0    81300.0  154.429993  188.759995  908.219971   
2024-05-08   178000.0    81300.0  153.619995  188.000000  911.469971   
2024-05-09   175400.0    79700.0  152.389999  189.500000  913.539978   
2024-05-10   179900.0    79200.0  151.919998  187.479996  930.289978   
2024-05-13   183800.0    78400.0  150.559998  186.570007  917.239990   
2024-05-14   185300.0    78300.0  153.160004  187.070007  915.030029   
2024-05-15        NaN        NaN  159.669998  185.990005  937.419983   

Ticker            GOOG        MSFT          MU        NVDA         TSM  \
Date                                                                     
2024-05-06  169.830002  413.540009  120.129997  921.400024  142.830002   
2024-05-07  172.979996  409.339996  119.209999  905.53997

In [8]:
# Set url to retrieve data
url_ffr = 'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey=WTH3FILIUYV3ONU8'

# Retrieve data
r = requests.get(url_ffr)
ffr_data = r.json()

In [9]:
# Check keys
print(ffr_data.keys())

dict_keys(['name', 'interval', 'unit', 'data'])


In [10]:
# Convert json into a DataFrame
ffr_df = pd.DataFrame(ffr_data['data'])

# Print to check
print(ffr_df)
print(ffr_df.info())

             date  value
0      2024-05-14   5.33
1      2024-05-13   5.33
2      2024-05-12   5.33
3      2024-05-11   5.33
4      2024-05-10   5.33
...           ...    ...
25516  1954-07-05   0.88
25517  1954-07-04  1.250
25518  1954-07-03  1.250
25519  1954-07-02  1.250
25520  1954-07-01   1.13

[25521 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25521 entries, 0 to 25520
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    25521 non-null  object
 1   value   25521 non-null  object
dtypes: object(2)
memory usage: 398.9+ KB
None


In [11]:
# Convert Date to datetime format
ffr_df['date'] = pd.to_datetime(ffr_df['date'])

# Rename columns
ffr_df.rename(columns={'date':'Date','value':'Interest Rate'},inplace=True)

# Print to check
print(ffr_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25521 entries, 0 to 25520
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           25521 non-null  datetime64[ns]
 1   Interest Rate  25521 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 398.9+ KB
None


In [12]:
# Merge Interest Rate column into new_df
new_df = pd.merge(new_df, ffr_df, on='Date', how='left')

# Print to check
print(new_df.info())
print(new_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           8 non-null      datetime64[ns]
 1   000660.KS      6 non-null      float64       
 2   005930.KS      6 non-null      float64       
 3   AMD            8 non-null      float64       
 4   AMZN           8 non-null      float64       
 5   ASML           8 non-null      float64       
 6   GOOG           8 non-null      float64       
 7   MSFT           8 non-null      float64       
 8   MU             8 non-null      float64       
 9   NVDA           8 non-null      float64       
 10  TSM            8 non-null      float64       
 11  ^GSPC          8 non-null      float64       
 12  ^IXIC          8 non-null      float64       
 13  ^SOX           8 non-null      float64       
 14  Interest Rate  7 non-null      object        
dtypes: datetime64[ns](1), float

In [13]:
# Set url to retrieve data
url_sma50 = f'https://www.alphavantage.co/query?function=SMA&symbol=NVDA&interval=daily&time_period=50&series_type=open&apikey=WTH3FILIUYV3ONU8'

#Retrieve data
r = requests.get(url_sma50)
sma50_data = r.json()

In [14]:
# Check keys
print(sma50_data.keys())

dict_keys(['Meta Data', 'Technical Analysis: SMA'])


In [15]:
# Convert json into a DataFrame
sma50_df = pd.DataFrame(sma50_data['Technical Analysis: SMA'])

# Print to check
print(sma50_df)
print(sma50_df.info())

    2024-05-15 2024-05-14 2024-05-13 2024-05-10 2024-05-09 2024-05-08  \
SMA   882.6561   881.2157   880.1211   878.0248   875.7819   873.1994   

    2024-05-07 2024-05-06 2024-05-03 2024-05-02  ... 2000-01-25 2000-01-24  \
SMA   871.1784   868.8980   867.1773   864.6238  ...     0.7754     0.7699   

    2000-01-21 2000-01-20 2000-01-19 2000-01-18 2000-01-14 2000-01-13  \
SMA     0.7632     0.7570     0.7500     0.7439     0.7375     0.7311   

    2000-01-12 2000-01-11  
SMA     0.7248     0.7173  

[1 rows x 6125 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, SMA to SMA
Columns: 6125 entries, 2024-05-15 to 2000-01-11
dtypes: object(6125)
memory usage: 47.9+ KB
None


In [16]:
# Melt sma50_df
sma50_df = pd.melt(sma50_df)

# Print to check
print(sma50_df)
print(sma50_df.info())

        variable     value
0     2024-05-15  882.6561
1     2024-05-14  881.2157
2     2024-05-13  880.1211
3     2024-05-10  878.0248
4     2024-05-09  875.7819
...          ...       ...
6120  2000-01-18    0.7439
6121  2000-01-14    0.7375
6122  2000-01-13    0.7311
6123  2000-01-12    0.7248
6124  2000-01-11    0.7173

[6125 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6125 entries, 0 to 6124
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   variable  6125 non-null   object
 1   value     6125 non-null   object
dtypes: object(2)
memory usage: 95.8+ KB
None


In [17]:
# Rename columns
sma50_df.rename(columns={'variable':'Date','value':'50D SMA'},inplace=True)

# Convert Date to datetime format
sma50_df['Date'] = pd.to_datetime(sma50_df['Date'])

# Print to check
print(sma50_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6125 entries, 0 to 6124
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     6125 non-null   datetime64[ns]
 1   50D SMA  6125 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 95.8+ KB
None


In [18]:
# Set url to retrieve data
url_sma200 = f'https://www.alphavantage.co/query?function=SMA&symbol=NVDA&interval=daily&time_period=200&series_type=open&apikey=WTH3FILIUYV3ONU8'

# Retrieve data
r = requests.get(url_sma200)
sma200_data = r.json()

In [19]:
# Convert json into a DataFrame
sma200_df = pd.DataFrame(sma200_data['Technical Analysis: SMA'])

# Print to check
print(sma200_df)
print(sma200_df.info())

    2024-05-15 2024-05-14 2024-05-13 2024-05-10 2024-05-09 2024-05-08  \
SMA   606.6390   604.3526   602.2055   600.0071   597.7924   595.5125   

    2024-05-07 2024-05-06 2024-05-03 2024-05-02  ... 2000-08-28 2000-08-25  \
SMA   593.2744   591.0084   588.8638   586.8470  ...     1.6549     1.6432   

    2000-08-24 2000-08-23 2000-08-22 2000-08-21 2000-08-18 2000-08-17  \
SMA     1.6314     1.6200     1.6093     1.5980     1.5854     1.5737   

    2000-08-16 2000-08-15  
SMA     1.5622     1.5500  

[1 rows x 5975 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, SMA to SMA
Columns: 5975 entries, 2024-05-15 to 2000-08-15
dtypes: object(5975)
memory usage: 46.7+ KB
None


In [20]:
# Melt sma200_df
sma200_df = pd.melt(sma200_df)

# Print to check
print(sma200_df)
print(sma200_df.info())

        variable     value
0     2024-05-15  606.6390
1     2024-05-14  604.3526
2     2024-05-13  602.2055
3     2024-05-10  600.0071
4     2024-05-09  597.7924
...          ...       ...
5970  2000-08-21    1.5980
5971  2000-08-18    1.5854
5972  2000-08-17    1.5737
5973  2000-08-16    1.5622
5974  2000-08-15    1.5500

[5975 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5975 entries, 0 to 5974
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   variable  5975 non-null   object
 1   value     5975 non-null   object
dtypes: object(2)
memory usage: 93.5+ KB
None


In [21]:
# Rename columns
sma200_df.rename(columns={'variable':'Date','value':'200D SMA'},inplace=True)

# Convert Date to datetime format
sma200_df['Date'] = pd.to_datetime(sma200_df['Date'])

# Print to check
print(sma200_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5975 entries, 0 to 5974
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      5975 non-null   datetime64[ns]
 1   200D SMA  5975 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 93.5+ KB
None


In [22]:
# Merge 50d SMA and 200d SMA columns into new_df
new_df = pd.merge(new_df, sma50_df, on='Date', how='left')
new_df = pd.merge(new_df, sma200_df, on='Date', how='left')

# Print to check
print(new_df.info())
print(new_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           8 non-null      datetime64[ns]
 1   000660.KS      6 non-null      float64       
 2   005930.KS      6 non-null      float64       
 3   AMD            8 non-null      float64       
 4   AMZN           8 non-null      float64       
 5   ASML           8 non-null      float64       
 6   GOOG           8 non-null      float64       
 7   MSFT           8 non-null      float64       
 8   MU             8 non-null      float64       
 9   NVDA           8 non-null      float64       
 10  TSM            8 non-null      float64       
 11  ^GSPC          8 non-null      float64       
 12  ^IXIC          8 non-null      float64       
 13  ^SOX           8 non-null      float64       
 14  Interest Rate  7 non-null      object        
 15  50D SMA        8 non-null  

In [23]:
# Forward fill nulls
new_df = new_df.ffill(axis=0)

# Print to check
print(new_df.info())
print(new_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           8 non-null      datetime64[ns]
 1   000660.KS      7 non-null      float64       
 2   005930.KS      7 non-null      float64       
 3   AMD            8 non-null      float64       
 4   AMZN           8 non-null      float64       
 5   ASML           8 non-null      float64       
 6   GOOG           8 non-null      float64       
 7   MSFT           8 non-null      float64       
 8   MU             8 non-null      float64       
 9   NVDA           8 non-null      float64       
 10  TSM            8 non-null      float64       
 11  ^GSPC          8 non-null      float64       
 12  ^IXIC          8 non-null      float64       
 13  ^SOX           8 non-null      float64       
 14  Interest Rate  8 non-null      object        
 15  50D SMA        8 non-null  

In [24]:
# Create list of column names without date
column_names = list(new_df.select_dtypes(include=['float64', 'object']))

# Create columns for previous day
for i, column in enumerate(column_names):
    new_df[f'{column}_pvd'] = new_df[column].shift(1)

# Print to check
print(new_df.info())
display(new_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8 non-null      datetime64[ns]
 1   000660.KS          7 non-null      float64       
 2   005930.KS          7 non-null      float64       
 3   AMD                8 non-null      float64       
 4   AMZN               8 non-null      float64       
 5   ASML               8 non-null      float64       
 6   GOOG               8 non-null      float64       
 7   MSFT               8 non-null      float64       
 8   MU                 8 non-null      float64       
 9   NVDA               8 non-null      float64       
 10  TSM                8 non-null      float64       
 11  ^GSPC              8 non-null      float64       
 12  ^IXIC              8 non-null      float64       
 13  ^SOX               8 non-null      float64       
 14  Interest Rate 

,Date,000660.KS,005930.KS,AMD,AMZN,ASML,GOOG,MSFT,MU,NVDA,...,MSFT_pvd,MU_pvd,NVDA_pvd,TSM_pvd,^GSPC_pvd,^IXIC_pvd,^SOX_pvd,Interest Rate_pvd,50D SMA_pvd,200D SMA_pvd
0,2024-05-06,NaN,NaN,155.779999,188.699997,916.919983,169.830002,413.540009,120.129997,921.400024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,2024-05-07,179600.0,81300.0,154.429993,188.759995,908.219971,172.979996,409.339996,119.209999,905.539978,...,413.540009,120.129997,921.400024,142.830002,5180.740234,16349.250000,4820.450195,5.33,868.8980,591.0084
2,2024-05-08,178000.0,81300.0,153.619995,188.000000,911.469971,171.160004,410.540009,119.320000,904.119995,...,409.339996,119.209999,905.539978,141.110001,5187.700195,16332.559570,4784.879883,5.33,871.1784,593.2744
3,2024-05-09,175400.0,79700.0,152.389999,189.500000,913.539978,171.580002,412.320007,117.809998,887.469971,...,410.540009,119.320000,904.119995,143.600006,5187.669922,16302.759766,4790.379883,5.33,873.1994,595.5125
4,2024-05-10,179900.0,79200.0,151.919998,187.479996,930.289978,170.289993,414.739990,121.239998,898.780029,...,412.320007,117.809998,887.469971,142.789993,5214.080078,16346.259766,4759.770020,5.33,875.7819,597.7924


In [25]:
# Create list of columns to drop 
column_names.remove('NVDA')

# Drop columns 
new_df = new_df.drop(column_names,axis=1)

# Print to check
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               8 non-null      datetime64[ns]
 1   NVDA               8 non-null      float64       
 2   000660.KS_pvd      6 non-null      float64       
 3   005930.KS_pvd      6 non-null      float64       
 4   AMD_pvd            7 non-null      float64       
 5   AMZN_pvd           7 non-null      float64       
 6   ASML_pvd           7 non-null      float64       
 7   GOOG_pvd           7 non-null      float64       
 8   MSFT_pvd           7 non-null      float64       
 9   MU_pvd             7 non-null      float64       
 10  NVDA_pvd           7 non-null      float64       
 11  TSM_pvd            7 non-null      float64       
 12  ^GSPC_pvd          7 non-null      float64       
 13  ^IXIC_pvd          7 non-null      float64       
 14  ^SOX_pvd      

In [26]:
# Drop nulls 
new_df = new_df.dropna()

# Print to check
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 2 to 7
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               6 non-null      datetime64[ns]
 1   NVDA               6 non-null      float64       
 2   000660.KS_pvd      6 non-null      float64       
 3   005930.KS_pvd      6 non-null      float64       
 4   AMD_pvd            6 non-null      float64       
 5   AMZN_pvd           6 non-null      float64       
 6   ASML_pvd           6 non-null      float64       
 7   GOOG_pvd           6 non-null      float64       
 8   MSFT_pvd           6 non-null      float64       
 9   MU_pvd             6 non-null      float64       
 10  NVDA_pvd           6 non-null      float64       
 11  TSM_pvd            6 non-null      float64       
 12  ^GSPC_pvd          6 non-null      float64       
 13  ^IXIC_pvd          6 non-null      float64       
 14  ^SOX_pvd           

In [27]:
# Check data statistics before Winsorisation
print(new_df.describe())

                      Date        NVDA  000660.KS_pvd  005930.KS_pvd  \
count                    6    6.000000       6.000000       6.000000   
mean   2024-05-11 12:00:00  909.036662  180333.333333   79700.000000   
min    2024-05-08 00:00:00  887.469971  175400.000000   78300.000000   
25%    2024-05-09 06:00:00  900.082520  178400.000000   78600.000000   
50%    2024-05-11 12:00:00  904.054993  179750.000000   79450.000000   
75%    2024-05-13 18:00:00  911.199997  182825.000000   80900.000000   
max    2024-05-15 00:00:00  946.299988  185300.000000   81300.000000   
std                    NaN   20.144375    3665.878703    1343.130671   

          AMD_pvd    AMZN_pvd    ASML_pvd    GOOG_pvd    MSFT_pvd      MU_pvd  \
count    6.000000    6.000000    6.000000    6.000000    6.000000    6.000000   
mean   152.679998  187.896667  915.964986  171.473330  412.870000  120.898332   
min    150.559998  186.570007  908.219971  170.289993  409.339996  117.809998   
25%    152.037498  187.1725

In [28]:
# Create list of columns to Winsorise
columns_win = ['NVDA', '50D SMA_pvd', '200D SMA_pvd',
               '000660.KS_pvd', 'AMD_pvd', 'MU_pvd',
               'NVDA_pvd']

# Apply Winsorisation
new_df[columns_win] = new_df[columns_win].apply(lambda x: winsorize(x, limits=[0, 0.03]))

# Print to check
print(new_df.describe())

                      Date        NVDA  000660.KS_pvd  005930.KS_pvd  \
count                    6    6.000000       6.000000       6.000000   
mean   2024-05-11 12:00:00  909.036662  180333.333333   79700.000000   
min    2024-05-08 00:00:00  887.469971  175400.000000   78300.000000   
25%    2024-05-09 06:00:00  900.082520  178400.000000   78600.000000   
50%    2024-05-11 12:00:00  904.054993  179750.000000   79450.000000   
75%    2024-05-13 18:00:00  911.199997  182825.000000   80900.000000   
max    2024-05-15 00:00:00  946.299988  185300.000000   81300.000000   
std                    NaN   20.144375    3665.878703    1343.130671   

          AMD_pvd    AMZN_pvd    ASML_pvd    GOOG_pvd    MSFT_pvd      MU_pvd  \
count    6.000000    6.000000    6.000000    6.000000    6.000000    6.000000   
mean   152.679998  187.896667  915.964986  171.473330  412.870000  120.898332   
min    150.559998  186.570007  908.219971  170.289993  409.339996  117.809998   
25%    152.037498  187.1725

In [29]:
# Export new_df to csv
new_df.to_csv('newdata.csv')